# Stitching Notebook

In [1]:
## Import library

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
import dask.dataframe as da

In [2]:
filename='pv_grid_variables.xlsx'
directory=''

In [3]:
## Get pv related variables from filename
df_data = pd.read_excel(directory+filename,sheet_name='coords')

In [4]:
## Construct unit and coord dictionaries with units and dimensions
unit_dict = dict(zip(df_data['var_name'],df_data['units']))
df_coord = pd.read_excel(directory + filename, sheet_name='coords')
unit_dict.update(dict(zip(df_coord['var_name'],df_coord['units'])))
num_dict = dict(zip(df_data['var_name'],df_data['number']))

In [5]:
## Initialize encoding dictionary
encoding_dict = {}
encoding_val = {'zlib':True,'_FillValue':0.0}

In [6]:
grid =  xr.Dataset()
filename_netCDF = '__J9GOH9T31G_singlePoint.nc'
sample = xr.open_dataset(filename_netCDF, decode_cf=False)
ori = ['lat', 'lon', 'ls', 'hr']
ori_dict = dict(zip(ori,[np.arange(-90,91,10), np.arange(-180,181,10), np.array([0,180]), np.array([10])]))
# ori_dict = dict(zip(ori,[np.arange(-90,91,10), np.arange(-180,181,10), np.arange(0,360,15), np.arange(0,25,2)]))

for vari in df_coord['var_name']:
    if vari in ori:
        grid[vari] = ori_dict[vari]
    else:
        grid[vari] = sample.coords[vari].values
    grid.coords[vari].attrs['units'] = unit_dict[vari]
    encoding_dict[vari] = encoding_val

# test = grid.to_dask_dataframe()

In [7]:
df_data = pd.read_excel(directory+filename,sheet_name='data')
unit_dict = dict(zip(df_data['var_name'],df_data['units']))
coord_dict = dict(zip(df_data['var_name'],df_data['dims']))
for vari in df_data['var_name']:
    coords = coord_dict[vari].split(',')
    zi = [len(grid.coords[i]) for i in coords]
    grid[vari] = (tuple(coords), np.zeros(zi,dtype=np.uint8))
    grid[vari].attrs['units'] = unit_dict[vari]
    encoding_dict[vari] = encoding_val

In [8]:
# da = grid.to_dask_dataframe()

grid.to_netcdf('PV_grid2.nc',encoding=encoding_dict)

# grid.to_hdf5('PV_grid2.hdf5',encoding=encoding_dict)

In [9]:
# grid.chunk()
#
# grid.to_netcdf('PV_grid_test2.nc',encoding=encoding_dict)

In [10]:
# a = grid.chunk()